In [1]:
import csv
import pandas as pd
import random
import pyarrow.feather as feather
import regex as re
from cleantext import clean
import pyarrow.feather as feather
from multiprocessing import Pool
import gc
import nltk
import itertools
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import *
import time
import gc

In [2]:
sampled_data = pd.read_csv("data/sample.csv", on_bad_lines='skip')
#Loads the sample that is 10% of the full dataset

In [3]:
len(sampled_data)

1155872

In [4]:
biases = sampled_data[sampled_data['type'] == '” said Linton Roberts']
biases1 = biases.reset_index()
biases1.content[4]

' Cammell Laird."'

In [5]:
sputnik = biases1[biases1['content'] == 'It only takes a few moments to share an article, but the person on the other end who reads it might have his life changed forever.']
len(sputnik)

0

In [6]:
def removeLabels(df): 
    df.type = df.type.replace({'political': 'reliable', 'junksci': 'fake', 'bias' : 'fake', 'satire': 'fake', 'conspiracy': 'fake', 'rumor': 'fake', 'unreliable' : 'fake', 'clickbait': 'fake', 'hate': 'fake'})
    df = df[(df.type == 'reliable') |(df.type == 'fake')]
    df = df[df.type.notnull()]
    df = df[df.content.notnull()]
    df = df[df.content != 'Dear readers, we are excited to announce that the Voice of Russia has changed its name and moved over to a new website. We are now known as Sputnik news agency and radio. You can find all the latest stories and updates here: http://sputniknews.com. Please update your bookmarks and stay with us!']
    df = df[df.content != 'It only takes a few moments to share an article, but the person on the other end who reads it might have his life changed forever.']

    return df

def labelperc(df): 
    labeldict = {}
    for i in df.type: 
        if i in labeldict: 
            labeldict[i] +=1
        else: 
            labeldict[i] = 1
    for i in labeldict: 
        labeldict[i] = labeldict[i]/len(df)*100
    return labeldict
cleanSample = removeLabels(sampled_data)
print(labelperc(cleanSample)) 
print(len(cleanSample))




{'reliable': 51.73583690219142, 'fake': 48.264163097808584}
838270


In [7]:
cleanSample.content[5]

'Cuban Province Well on Its Way to 100% Renewable Energy'

In [8]:
def checkForDuplicateArticles(df): 
    articledict = {}
    df_sample = df.sample(frac=0.1, random_state=1)
    for i in df_sample.content: 
        if i in articledict: 
            articledict[i] +=1
        else: 
            articledict[i] = 1
    return articledict
    

print(checkForDuplicateArticles(cleanSample))
